# FactSales – Silver → Gold

**Objetivo:** Crear la tabla de hechos `FactSales` en Gold.  

**Proceso:**
1. Leer los pedidos de `Orders` en Silver.  
2. Sustituir los campos de fecha (`OrderDate`, `DueDate`, `ShipDate`) por claves foráneas a la dimensión de fechas (`DimFecha`).  
3. Resolver claves foráneas con las dimensiones: Cliente, Empleado, Producto, Proveedor, Categoría, Subcategoría.  
4. Cargar el resultado en `Tables/FactSales` en Gold.  


In [1]:
%%sql
CREATE OR REPLACE TEMP VIEW tmpFactSales AS
SELECT
    o.SalesOrderID AS SalesOrderKey,
    IFNULL(c.CustomerKey, -1) AS CustomerKey,
    IFNULL(e.EmployeeKey, -1) AS EmployeeKey,
    IFNULL(p.ProductKey, -1) AS ProductKey,
    IFNULL(vp.VendorID, -1) AS VendorKey,
    IFNULL(psc.CategoryKey, -1) AS CategoryKey,
    IFNULL(psc.SubCategoryKey, -1) AS SubCategoryKey,
    
    -- Claves de fechas
    dfOrder.DateKey   AS OrderDateKey,
    dfDue.DateKey     AS DueDateKey,
    dfShip.DateKey    AS ShipDateKey,
    
    -- Métricas de negocio    
    o.SubTotal,
    o.TaxAmt,
    o.Freight,
    o.TotalDue,
    o.OrderQty,
    o.UnitPrice,
    o.UnitPriceDiscount,
    o.LineTotal,
    o.ETL_DateInserted
    
FROM AdventureWorks_SilverLayer.AdventureWorks_Silver_Orders o

-- Dimensiones
LEFT JOIN AdventureWorks_GoldLayer.DimCustomer c       ON c.CustomerKey = o.CustomerID
LEFT JOIN AdventureWorks_GoldLayer.DimEmployee e       ON e.EmployeeKey = o.EmployeeID
LEFT JOIN AdventureWorks_GoldLayer.DimProduct p        ON p.ProductKey  = o.ProductID
LEFT JOIN AdventureWorks_GoldLayer.DimProductSubCategory psc ON psc.SubCategoryKey = p.SubCategoryID

-- Vendor (muchos a muchos por VendorProduct)
LEFT JOIN (
   SELECT ProductID, VendorID
   FROM AdventureWorks_SilverLayer.AdventureWorks_Silver_VendorProduct
   GROUP BY ProductID, VendorID
) vp ON vp.ProductID = p.ProductKey

-- DimFecha (se une por igualdad de fecha)
LEFT JOIN AdventureWorks_GoldLayer.DimFecha dfOrder ON dfOrder.FullDate = o.OrderDate
LEFT JOIN AdventureWorks_GoldLayer.DimFecha dfDue   ON dfDue.FullDate   = o.DueDate
LEFT JOIN AdventureWorks_GoldLayer.DimFecha dfShip  ON dfShip.FullDate  = o.ShipDate

StatementMeta(, fbe924d9-e0e3-47bf-88e4-4195c0888011, 2, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [2]:
# Guardar FactSales en Gold
df = spark.sql("SELECT * FROM tmpFactSales")

# Verificar primeras filas antes de guardar
#df.show(10)  

## Guardar la tabla FactSales en Gold (Delta)
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/FactSales")


StatementMeta(, fbe924d9-e0e3-47bf-88e4-4195c0888011, 4, Finished, Available, Finished)

+-------------+-----------+-----------+----------+---------+-----------+--------------+------------+----------+-----------+-----------+---------+---------+-----------+--------+---------+-----------------+---------+--------------------+
|SalesOrderKey|CustomerKey|EmployeeKey|ProductKey|VendorKey|CategoryKey|SubCategoryKey|OrderDateKey|DueDateKey|ShipDateKey|   SubTotal|   TaxAmt|  Freight|   TotalDue|OrderQty|UnitPrice|UnitPriceDiscount|LineTotal|    ETL_DateInserted|
+-------------+-----------+-----------+----------+---------+-----------+--------------+------------+----------+-----------+-----------+---------+---------+-----------+--------+---------+-----------------+---------+--------------------+
|        47065|       1241|        276|       863|     1676|          3|            20|    20120630|  20120712|   20120707| 55971.0000|5395.0000|1686.0000| 63052.0000|      34|  19.0000|           0.0000| 581.0000|2025-10-02 18:52:...|
|        47400|        977|        276|       863|     1